In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Grid search

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [13]:
 def train_RF(n_est, depth): # Create loop for random forest classifier with grid search
        rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
        rf_model = rf.fit(X_train, y_train)
        y_pred = rf_model.predict(X_test)
        precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
        print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [14]:
for en_est in [10, 50, 100]: # Loop used in grid search with proposed en_est and depth
    for depth in [10, 20, 30, None]:
        train_RF(en_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.24 / Accuracy: 0.898
Est: 10 / Depth: 20 ---- Precision: 0.958 / Recall: 0.607 / Accuracy: 0.943
Est: 10 / Depth: 30 ---- Precision: 0.969 / Recall: 0.833 / Accuracy: 0.974
Est: 10 / Depth: None ---- Precision: 1.0 / Recall: 0.813 / Accuracy: 0.975
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.273 / Accuracy: 0.902
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.587 / Accuracy: 0.944
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.753 / Accuracy: 0.967
Est: 50 / Depth: None ---- Precision: 0.984 / Recall: 0.82 / Accuracy: 0.974
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.253 / Accuracy: 0.899
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.653 / Accuracy: 0.953
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.76 / Accuracy: 0.968
Est: 100 / Depth: None ---- Precision: 0.984 / Recall: 0.833 / Accuracy: 0.976
